# Exercise: Black Scholes Fusion

The Black Scholes model provides a fantastic example of a real-world set of equations that greatly benefits from operator fusion. Black Scholes provides both a complex set of expressions that provide significant readability improvements if expressed as individual expressions, but also benefits from fusion of its separate operational parts. Below is a brief description of the Black Scholes models and its composite terms:


$$
C(S_0, K, T) = S_0 \,\Phi\bigl(d_1\bigr) \;-\; K \, e^{-rT} \,\Phi\bigl(d_2\bigr),
$$

where

$$
d_1 = \frac{\ln\!\bigl(\tfrac{S_0}{K}\bigr) + \bigl(r + \tfrac{\sigma^2}{2}\bigr)T}{\sigma \sqrt{T}},
\quad
d_2 = d_1 - \sigma \sqrt{T}.
$$

Here:
- \( S_0 \) is the current stock price
- \( K \) is the strike price
- \( T \) is the time to maturity (in years)
- \( r \) is the risk-free interest rate (annualized)
- \( \sigma \) is the volatility of the underlying stock (annualized)
- \( \Phi(\cdot) \) is the cumulative distribution function (CDF) of the standard normal distribution



We can easily translate this by expressing each of the terms defined above as separate MatX operators, then fusing the execution of those operators in the final run command.

Try breaking the equation below into the following operators:

```
VsqrtT  = V * sqrt(T);
d1      = (log(S / K) + (r + 0.5 * V * V) * T) / VsqrtT ;
d2      = d1 - VsqrtT;
cdf_d1  = normcdf(d1);
cdf_d2  = normcdf(d2);
expRT   = exp(-1 * r * T); 
```

In [ ]:
%%run_matx
auto exec = matx::CUDAExecutor();

using dtype = double;
matx::index_t input_size = 100;


// ---- declare input data ---- //
auto K = matx::make_tensor<dtype>({input_size});
auto S = matx::make_tensor<dtype>({input_size});
auto V = matx::make_tensor<dtype>({input_size});
auto r = matx::make_tensor<dtype>({input_size});
auto T = matx::make_tensor<dtype>({input_size});
auto output = matx::make_tensor<dtype>({input_size});  

// ---- populate the data ---- //
(K = matx::random<float>(K.Shape(), matx::NORMAL)).run();
(S = matx::random<float>(S.Shape(), matx::NORMAL)).run();
(V = matx::random<float>(V.Shape(), matx::NORMAL)).run();
(r = matx::random<float>(r.Shape(), matx::NORMAL)).run();
(T = matx::random<float>(T.Shape(), matx::NORMAL)).run();
(output = matx::zeros({input_size})).run(exec);
exec.sync();

auto VsqrtT = V * sqrt(T);
auto d1     = (log(S / K) + (r + 0.5 * V * V) * T) / VsqrtT ;
auto d2     = d1 - VsqrtT;
auto cdf_d1 = matx::normcdf(d1);
auto cdf_d2 = matx::normcdf(d2);
auto expRT  = exp(-1 * r * T); 
exec.start_timer();
(output = S * cdf_d1 - K * expRT * cdf_d2).run(exec);
exec.stop_timer();
std::cout <<"Fused Runtime: " << exec.get_time_ms() << " ms" << std::endl;